#### Model 1: to have tax rules in the separate contracts.

Virtual currency transactions are taxable by law just like transactions in any other property. The IRS has issued guidance in IRS Notice 2014-21 for use by taxpayers and their return preparers that addresses transactions in virtual currency, also known as digital currency. We will use it for illustration of tax rules for smart contracts.

Notice 2014-21 provides that virtual currency is treated as property for U.S. federal tax purposes. General tax principles that apply to property transactions apply to transactions using virtual currency. Among other things, this means that:

    A payment made using virtual currency is subject to information reporting to the same extent as any other payment made in property.
    
    Payments using virtual currency made to independent contractors and other service providers are taxable, and self-employment tax rules generally apply.  Normally, payers must issue Form 1099-MISC.
    
    Wages paid to employees using virtual currency are taxable to the employee, must be reported by an employer on a Form W-2 and are subject to federal income tax withholding and payroll taxes.
    
    Certain third parties who settle payments made in virtual currency on behalf of merchants that accept virtual currency from their customers are required to report payments to those merchants on Form 1099-K, Payment Card and Third Party Network Transactions.

    The character of gain or loss from the sale or exchange of virtual currency depends on whether the virtual currency is a capital asset in the hands of the taxpayer.

#### Required attributes to construct tax clouse

In general, the sale or exchange of convertible virtual currency, or the use of convertible virtual currency to pay for goods or services in a real-world economy transaction, has tax consequences that may result in a tax liability:

- is it convertible virtual currency (bool);

A taxpayer who receives virtual currency as payment for goods or services must, in computing gross income, include the fair market value of the virtual currency, measured in U.S. dollars, as of the date that the virtual currency was received:

- fair market value of the virtual currency (int);

If the fair market value of property received in exchange for virtual currency is higher / lower the taxpayer’s adjusted basis of the virtual currency, the taxpayer has taxable gain / loss:

- fair market value of property received (int);

A payment made using virtual currency is subject to information reporting to the same extent as any other payment made in property. For example, a person who in the course of a trade or business makes a payment of fixed and determinable income using virtual currency with a value of $600 or more to a U.S. non-exempt recipient in a taxable year is required to report the payment to the IRS and to the payee. Examples of payments of fixed and determinable income include rent, salaries, wages, premiums, annuities, and compensation:

- reportable event (bool);

Payments made using virtual currency are subject to backup withholding to the same extent as other payments made in property. Therefore, payors making reportable payments using virtual currency must solicit a taxpayer identification number (TIN) from the payee. The payor must backup withhold from the payment if a TIN is not obtained prior to payment or if the payor receives notification from the IRS that backup withholding is required:

- TIN (int);


etc.



#### The tax consequences schould be assessed before payment (and after payment):

- there might be obligation to withold tax before payment;
- after assessment of tax consequences the payment might be not desired or can be structured in a different way;

etc.

Therefore the tax clause is beneficial to structure as a separate contract and that contract schould be deployed before the main contract. The same approcah might be considered for other types of clauses as well.

Such approach allows code reusability, ie treating deployed contracts like libraries. And as some tax rules are stable and some are frequently changes, than practical way might be to use one separate callable contract per rule.

In [10]:
import json
import web3

from web3 import Web3
from solc import compile_source
from web3.contract import ConciseContract

In [11]:
# Solidity source code
contract_source_code = '''
pragma solidity ^0.4.21;

//Contract to store tax rate
contract TaxRate{
    uint private taxRate = 20;
    function getRate() public view returns (uint){
        return taxRate;
    }
    
    function setRate(uint _rate) public{
        taxRate = _rate;
    }
}

contract Base1 {

    //Tax procedures
    function useNewRate() public returns (uint){
    
        TaxRate myObj = new TaxRate();
        myObj.setRate(10);
        return myObj.getRate();
    }
    
    //Base function
    function withdraw() public {
        msg.sender.transfer(this.balance);
    }   
}

'''

In [12]:
compiled_sol = compile_source(contract_source_code) # Compiled source code
contract_interface = compiled_sol['<stdin>:Base1']

# web3.py instance
w3 = Web3(Web3.EthereumTesterProvider())

# set pre-funded account as sender
w3.eth.defaultAccount = w3.eth.accounts[0]

# Instantiate and deploy contract
Base = w3.eth.contract(abi=contract_interface['abi'], bytecode=contract_interface['bin'])

# Submit the transaction that deploys the contract
tx_hash = Base.constructor().transact()

# Wait for the transaction to be mined, and get the transaction receipt
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

# Create the contract instance with the newly-deployed address
base = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=contract_interface['abi'],
)

# Display the default  from the contract
print('New contract rate: {}'.format(
    base.functions.useNewRate().call()
))

New contract rate: 10


In [14]:
def get_contract_interface(input_str, contract_source_code):
    # Solidity Compiler

    compiled_sol = compile_source(contract_source_code) # Compiled source code
    contract_interface = compiled_sol['<stdin>:' + input_str]

    # web3.py instance
    w3 = Web3(Web3.EthereumTesterProvider())

    # set pre-funded account as sender
    w3.eth.defaultAccount = w3.eth.accounts[0]

    # Instantiate and deploy contract
    ContractDeploy = w3.eth.contract(abi=contract_interface['abi'], bytecode=contract_interface['bin'])

    # Submit the transaction that deploys the contract
    tx_hash = ContractDeploy.constructor().transact()

    # Wait for the transaction to be mined, and get the transaction receipt
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

    # Create the contract instance with the newly-deployed address
    contract_inst = w3.eth.contract(
        address=tx_receipt.contractAddress,
        abi=contract_interface['abi'],
    )
    
    return w3, contract_inst